# Logistic Regression

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

pd.options.display.max_columns = None

**Resources**
- [5 Reasons “Logistic Regression” should be the first thing you learn when becoming a Data Scientist](https://towardsdatascience.com/5-reasons-logistic-regression-should-be-the-first-thing-you-learn-when-become-a-data-scientist-fcaae46605c4)
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - more code and math, with gradient descent and the logistic loss function
- more classification models from scikit-learn: [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math varies significantly, but the API and interpretation are fairly similar
- More on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), how audio features were actually calculated in the FMA dataset used at the end of this notebook

There are many options for regression. Recall that **linear regression** is helpful when predicting a real number along some continuum without a restricted range. Things like income and age.

But what about . . .
* probabilities
* binary values (sick vs. healthy, male vs. female)
* approval rating (0% to 100%)

We need some "squishification" process to map numbers from a real number continuum to the unit interval (X range $-\infty$ to $\infty$, Y range 0 to 1). We need a **cumulative density function** - most commonly the **[sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function)**. This function in particular is useful because its calculous properties give it nice behaviors. (e^x derivative is e^x) Another common CDF is the **[probit function](https://en.wikipedia.org/wiki/Probit)** for probabilities.

With logistic regression, we can even model general multinomial classification problems by combining several logistic regressions to predict membership in various classes and outputting the class that is most likely.

At its heart, logistic regression is still linear. That is, the underlying math and optimization follow traditional linear regression. However, our coefficients won't have intuitive linear interpretation.

$S(x) = \frac{1}{1 + e^{-x}} = \frac{e^x}{e^x + 1}$


**What data science methods are used at work?** (source: [Kaggle 2017 Survey](https://www.kaggle.com/surveys/2017))
![](img/kaggle-common-algos.PNG)

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Example 1: Linear vs. Logistic, Return of the Titanic 🚢

To pull Titanic data from Kaggle:
* create account
* generate API Key and `kaggle.json`
* join [Titanic competition](https://www.kaggle.com/c/titanic/data)

The [Kaggle API](https://github.com/Kaggle/kaggle-api) enables lots of convenient methods for interacting with Kaggle.

Here's how the data were sourced:
```python
!pip install kaggle

import os, json

path_to_config = "../kaggle.json"
kaggle_config = json.loads(open(path_to_config).read())
os.environ['KAGGLE_USERNAME'] = kaggle_config['username']
os.environ['KAGGLE_KEY'] = kaggle_config['key']

!kaggle competitions download -c titanic -p datasets/titanic
```

In [14]:
# How would we try to do this with linear regression?
train_df = pd.read_csv('datasets/titanic/train.csv').dropna()
test_df = pd.read_csv('datasets/titanic/train.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [34]:
predictors = ['Pclass', 'Age', 'Fare']
target = 'Survived'

X = train_df[predictors]
y = train_df[target]

linear_reg = LinearRegression().fit(X, y)
print('score:', linear_reg.score(X, y))

linear_reg.predict(test_df[predictors])[:10]

score: 0.08389810726550939


array([0.66567686, 0.68168731, 0.52351404, 0.74909452, 0.4779952 ,
       0.58445856, 0.73115482, 0.91675714, 0.57710857, 0.43722395])

How can you "partially" survive? You either survived or did not, like how our target column is encoded.

In [25]:
{attr: coef for attr, coef in zip(predictors, linear_reg.coef_)}

{'Age': -0.008293140798696792,
 'Fare': 0.00048775407963190954,
 'Pclass': -0.08596294986392504}

In [26]:
test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

How can survial be greater than 1?

In [33]:
log_reg = LogisticRegression().fit(X, y)
print(log_reg.score(X, y))

log_reg.predict(test_df[predictors])[:10]

0.7103825136612022


array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0], dtype=int64)

In [35]:
log_reg.predict(test_case)[0]

1

In [37]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [59]:
# What's the math?
{attr: coef for attr, coef in zip(predictors, log_reg.coef_[0])}

{'Age': -0.02912512677284113,
 'Fare': 0.004803700867242946,
 'Pclass': -0.04550170440859573}

In [39]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
    return 1 / (1 + np.e**(-x))

In [17]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

**Note:** while the sign (-/+) of coefficients provides information on the relationship between X and Y, we can't infer that a one-unit difference in X corresponds to a one-unit difference in Y.

## Example 2: Multinomial Logistic Regression

[Absenteeism at work dataset](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) has 21 classes. `sklearn.linear_model.LogisticRegression` automatically handles more than two classes by essentially treating each label as different (1) from some base class (0).

Here's how the data were sourced:


In [106]:
data_url = ('http://archive.ics.uci.edu/ml/'
            'machine-learning-databases/00445/'
            'Absenteeism_at_work_AAA.zip')

extract_zip_url(data_url, 'datasets/abs')

In [107]:
df = pd.read_csv('datasets/abs/Absenteeism_at_work.csv', sep=';')
pd.options.display.max_columns = df.shape[1]
df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2


In [78]:
def multinom_logistic(df, target, predictors, test_size=0.5, random_state=42):
    X = df[predictors]
    y = df[target]

    if test_size:
        X_train, X_test, Y_train, Y_test = (
            train_test_split(X, y, test_size=test_size, 
                             random_state=random_state)
        )
    else:
        X_train, X_test = [X] * 2
        Y_train, Y_test = [y] * 2
        
    model = LogisticRegression(random_state=random_state, solver='lbfgs', 
                               multi_class='multinomial', max_iter=5000)

    model.fit(X_train, Y_train)
    print('score:', model.score(X_test, Y_test))
    
    return model, X_test, Y_test

In [82]:
target = 'Reason for absence'
predictors = sorted(list(set(df.columns) - 
                         set(['ID', 'Reason for absence'])))

0.5121621621621621


In [100]:
# without train-test
model = multinom_logistic(df, target, predictors, test_size=None, random_state=42)

score: 0.5121621621621621


In [93]:
model = multinom_logistic(df, target, predictors, test_size=0.5, random_state=42)

score: 0.3810810810810811


In [96]:
# smaller test size makes worse score
model = multinom_logistic(df, target, predictors, test_size=0.2, random_state=42)

score: 0.3783783783783784


## Example 3: real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

In [3]:
# # downloads 1.36 GB
# data_url = 'https://os.unil.cloud.switch.ch/fma/fma_metadata.zip'
# extract_zip_url(data_url, 'datasets/fma', flatten=True)

In [47]:
headers = pd.read_csv('datasets/fma/tracks.csv', nrows=3, 
                      header=None)
cols = (headers.T.fillna('')
               .apply(lambda x: ('_'.join(x.astype(str))
                                    .strip('_')), axis=1))

tracks = pd.read_csv('datasets/fma/tracks.csv', skiprows=3, 
                     header=None, names=cols)

In [76]:
print(tracks.shape)
tracks.head()

(20980, 53)


,track_id,album_comments,album_date_created,album_date_released,album_engineer,album_favorites,album_id,album_information,album_listens,album_producer,album_tags,album_title,album_tracks,album_type,artist_active_year_begin,artist_active_year_end,artist_associated_labels,artist_bio,artist_comments,artist_date_created,artist_favorites,artist_id,artist_latitude,artist_location,artist_longitude,artist_members,artist_name,artist_related_projects,artist_tags,artist_website,artist_wikipedia_page,set_split,set_subset,track_bit_rate,track_comments,track_composer,track_date_created,track_date_recorded,track_duration,track_favorites,track_genre_top,track_genres,track_genres_all,track_information,track_interest,track_language_code,track_license,track_listens,track_lyricist,track_number,track_publisher,track_tags,track_title
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
9,134,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:43:19,2008-11-26 00:00:00,207,3,Hip-Hop,[21],[21],NaN,1126,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,943,NaN,5,NaN,[],Street Music
12,137,1,2008-11-26 01:49:35,2006-12-01 00:00:00,NaN,2,59,<p>Here's the proof in the pudding that the as...,1681,NaN,['lafms'],Live at LACE,2,Live Performance,1978-01-01 00:00:00,1998-01-01 00:00:00,"Los Angeles Free Music Society, Harbinger Sound",<p>Airway was a musical ensemble based within ...,0,2008-11-26 01:47:22,5,53,34.052234,"Los Angeles, CA",-118.243685,"Rick Potts, Juan Gomez, Tom Recchion, Joe Pott...",Airway,"Los Angeles Free Music Society, the Banshees, ...",['airway'],http://www.lafms.com/,http://en.wikipedia.org/wiki/Airway_(band),training,large,256000,0,NaN,2008-11-26 01:43:42,1978-04-27 00:00:00,1233,2,Experimental,"[1, 32]","[32, 1, 38]",<p>Recorded live in downtown Los Angeles at th...,2559,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1278,NaN,1,NaN,['lafms'],Side A


In [70]:
target = 'track_genre_top'
target_aliases = ['track_genres', 'track_genres_all']
predictors = ['track_duration', 'track_favorites', 'track_interest', 
              'track_listens', 'artist_latitude', 'artist_longitude']

In [69]:
tracks = tracks.dropna(subset=[target]+predictors)

In [79]:
model, X_test, Y_test = (
    multinom_logistic(tracks, target, predictors,
                      test_size=0.5, random_state=42)
)

score: 0.40896091515729266


In [80]:
from collections import Counter

predictions = model.predict(X_test)

d = {key:0 for key in set(Y_test)}
for actual, predict in zip(Y_test, predictions):
    if actual==predict:
        d[actual] += 1

counts = Counter(Y_test)

for key, value in d.items():
    d[key] = d[key] / counts[key]

prev = pd.Series(counts, name='% prevalence') / len(Y_test) * 100
correct = pd.Series(d, name='% correct') * 100

print(pd.DataFrame([prev, correct]).T.round(1))

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.